In [2]:
# notebook based largely on preprocessing in https://www.kaggle.com/maria17/cord-19-explore-drugs-being-developed

In [3]:
# imports - downloads

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import json
import csv
import os

from tqdm import tqdm_notebook as tqdm
from p_tqdm import p_map

import random
import nltk
import sys

from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer 
from nltk.tag import StanfordNERTagger
from nltk.parse.corenlp import CoreNLPDependencyParser
from nltk.parse import CoreNLPParser
from nltk.tokenize import sent_tokenize, word_tokenize


nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

import string

[nltk_data] Downloading package punkt to /home/chryssa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/chryssa/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/chryssa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/chryssa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
indir = 'datasets/CORD-19-research-challenge'

publications = []
for root, dirs, files in os.walk(indir):
    for file in files:
        if file.endswith('json'):
            path = os.path.join(root, file)
            publications.append(path)

        
print('Number of publications: %d'%len(publications))

Number of publications: 29315


In [ ]:
#process files

#selected columns: 
selected_columns = ['paper_id', 'metadata.title', 'body_text', 'abstract', ]
#is a dataframe what we need to keep info?

list_df = []
for pub in tqdm(publications):
    with open(pub) as jf:
        json_data = json.load(jf)
        json_data_df = pd.io.json.json_normalize(json_data)
        json_data_df_selected = json_data_df[selected_columns]

        list_df.append(json_data_df_selected)
publications_df = pd.concat(list_df)     



In [33]:
publications_df['abstract_text'] = publications_df['abstract'].apply(lambda x: x[0]['text'] if x else "")
publications_df['all_body_text'] = publications_df['body_text'].apply(lambda x: " ".join([(t['text']) for t in x]))
publications_df['title'] = publications_df['metadata.title']

print(publications_df.shape)
publications_df.head(3)
df_text = publications_df.drop(['metadata.title','abstract', 'body_text'], axis=1)
print(df_text.shape)
df_text.head(3)

(29315, 7)
(29315, 4)


,paper_id,abstract_text,all_body_text,title
0,0015023cc06b5362d332b3baf348d11567ca2fbb,word count: 194 22 Text word count: 5168 23 24...,"VP3, and VP0 (which is further processed to VP...",The RNA pseudoknots in foot-and-mouth disease ...
0,004f0f8bb66cf446678dc13cf2701feec4f36d76,,The 2019-nCoV epidemic has spread across China...,Healthcare-resource-adjusted vulnerabilities t...
0,00d16927588fb04d4be0e6b269fc02f0d3c2aa7b,Infectious bronchitis (IB) causes significant ...,"Infectious bronchitis (IB), which is caused by...","Real-time, MinION-based, amplicon sequencing f..."


In [35]:
def no_title(row):
    return not row.title.strip()

def no_abstract(row):
    return not row.abstract_text.strip()

def no_body(row):
    return not row.all_body_text.strip()

def no_title_abstract_body(row):
    return no_title(row) and no_abstract(row) and no_body(row)

mask = df_text.apply(no_title_abstract_body, axis=1)
print('Number of articles that have no text data at all:', df_text.loc[mask].shape[0])

Number of articles that have no text data at all: 0


In [1]:
#picklify dataset

df_text.to_pickle('covid_pub_dataset.pickle')

NameError: name 'df_text' is not defined

In [ ]:
df_text.loc[df_text.apply(no_title, axis=1), 'title'] = None
df_text.loc[df_text.apply(no_abstract, axis=1), 'abstract_text'] = None
df_text.loc[df_text.apply(no_body, axis=1), 'body_text'] = None


In [75]:
#define keywords
keywords_covid = ['sars-cov', 'sars', 'coronavirus',  'coronaviruses', 'ncov', 'covid-19', 'covid19', '2019-ncov', '2019-ncov.', 'wuhan', 'sars-cov-2']
keywords_risk = ['risk', 'risk-factor']

In [69]:


stpwds = stopwords.words('english')
punct = string.punctuation.replace('-', '')

#title_tokens = clean_text_simple(title,my_stopwords=stpwds,punct=punct, stemming=False)


def has_keyword(text, keywords):
    for key in keywords:
        if text and key.lower() in text.lower():
            return True
    return False

df_text['title_has_covid19'] = df_text.title.apply(has_keyword, args=[keywords_covid])
df_text['abstract_has_covid19'] = df_text.abstract_text.apply(has_keyword, args=[keywords_covid])
df_text['body_has_covid19'] = df_text.all_body_text.apply(has_keyword, args=[keywords_covid])
# df['body_has_covid19'] = df.body.apply(has_covid19)
df_covid19 = df_text[df_text.title_has_covid19 | df_text.abstract_has_covid19]
print(df_covid19.shape)

(5444, 9)


In [76]:
df_covid19['title_has_risk'] = df_covid19.title.apply(has_keyword, args=[keywords_risk])
df_covid19['abstract_has_risk'] = df_covid19.abstract_text.apply(has_keyword, args=[keywords_risk])
df_covid19['body_has_risk'] = df_covid19.all_body_text.apply(has_keyword, args=[keywords_risk])
# df['body_has_covid19'] = df.body.apply(has_covid19)
df_risk = df_covid19[df_covid19.title_has_risk | df_covid19.abstract_has_risk]
print(df_risk.shape)

/home/chryssa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/chryssa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(395, 12)


/home/chryssa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
def clean_text_simple(text, my_stopwords, punct, remove_stopwords=True, pos_filtering=True, stemming=True):
    text = text.lower()
    text = ''.join(l for l in text if l not in punct) # remove punctuation (preserving intra-word dashes)
    text = re.sub(' +',' ',text) # strip extra white space
    text = text.strip() # strip leading and trailing white space
    # tokenize (split based on whitespace)
    tokens = text.split(' ')
    if pos_filtering == True:
        # POS tag and retain only nouns and adjectives
        tagged_tokens = pos_tag(tokens)
        tokens_keep = []
        for item in tagged_tokens:
            if (
            item[1] == 'NN' or
            item[1] == 'NNS' or
            item[1] == 'NNP' or
            item[1] == 'NNPS' or
            item[1] == 'JJ' or
            item[1] == 'JJS' or
            item[1] == 'JJR'
            ):
                tokens_keep.append(item[0])
        tokens = tokens_keep
    if remove_stopwords:
        # remove stopwords
        tokens = [token for token in tokens if token not in my_stopwords]
    if stemming:
        # apply Porter's stemmer
        stemmer = nltk.stem.PorterStemmer()
        tokens_stemmed = list()
        for token in tokens:
            tokens_stemmed.append(stemmer.stem(token))
        tokens = tokens_stemmed
    
    return(tokens)